### QAOA SIZE 8,16 REPS 1,3 SEED 11,22,33 IN IBM (Falcon) AND QUANTINUUM

In [1]:
# Caller
from pathlib import Path
import sys

# Discover project root
proj = Path.cwd().resolve()
for _ in range(8):
    if (proj / "results" / "pairwise" / "summary").exists() and (proj / "notebooks").exists():
        break
    if proj.parent == proj:
        break
    proj = proj.parent

# Import helper
sys.path.insert(0, str(proj / "notebooks"))
from notebook_helper import load_family, tidy, show_table  # (sort_best_worst, add_rank_within_size available if needed)

# analysis parameters
FAMILY = "qaoa"  # or: "qaoa","vqe_two_local","qft","grover","vbe_adder","randomcircuit"
BACKENDS = ("ibm_falcon","quantinuum")

loaded = load_family(FAMILY, BACKENDS)

for backend, df in loaded.items():
    tbl = tidy(df)
    show_table(tbl, caption=f"{FAMILY.upper()} – {backend} (raw tidy)")

    # To rank later, uncomment:
    # from notebook_helper import sort_best_worst, add_rank_within_size
    # PRIMARY_METRIC = "depth_delta"  # or "n_ops_total_gates_delta", "n_ops_total_delta", etc.
    # ranked = sort_best_worst(tbl, primary=PRIMARY_METRIC, per_size=True)
    # ranked = add_rank_within_size(ranked, by=PRIMARY_METRIC)
    # show_table(ranked, caption=f"{FAMILY.upper()} – {backend} (ranked by {PRIMARY_METRIC})")

,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,ibm_falcon,qaoa,8,r3_seed33_sym,RB→RR,93,93,+0,270,270,+0,270,270,+0,186,186,+0,84,84,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rz"", 130], [""cx"", 84], [""sx"", 56]]","[[""rz"", 130], [""cx"", 84], [""sx"", 56]]",8,8,2025-10-05T22:22:49
1,ibm_falcon,qaoa,8,r3_seed33_sym,RR→RB,93,93,+0,270,270,+0,270,270,+0,186,186,+0,84,84,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rz"", 130], [""cx"", 84], [""sx"", 56]]","[[""rz"", 130], [""cx"", 84], [""sx"", 56]]",8,8,2025-10-05T22:22:49
2,ibm_falcon,qaoa,8,r3_seed33_sym,RB→CTM,93,92,-1,270,270,+0,270,270,+0,186,186,+0,84,84,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rz"", 130], [""cx"", 84], [""sx"", 56]]","[[""rz"", 130], [""cx"", 84], [""sx"", 56]]",8,8,2025-10-05T22:22:49
3,ibm_falcon,qaoa,8,r3_seed33_sym,CTM→RB,93,92,-1,270,270,+0,270,270,+0,186,186,+0,84,84,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rz"", 130], [""cx"", 84], [""sx"", 56]]","[[""rz"", 130], [""cx"", 84], [""sx"", 56]]",8,8,2025-10-05T22:22:49
4,ibm_falcon,qaoa,8,r3_seed33_sym,RR→CTM,93,92,-1,270,270,+0,270,270,+0,186,186,+0,84,84,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rz"", 130], [""cx"", 84], [""sx"", 56]]","[[""rz"", 130], [""cx"", 84], [""sx"", 56]]",8,8,2025-10-05T22:22:49
5,ibm_falcon,qaoa,8,r3_seed33_sym,CTM→RR,93,91,-2,270,261,-9,270,261,-9,186,177,-9,84,84,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rz"", 130], [""cx"", 84], [""sx"", 56]]","[[""rz"", 121], [""cx"", 84], [""sx"", 56]]",8,8,2025-10-05T22:22:49
6,ibm_falcon,qaoa,16,r3_seed22_sym,RB→RR,150,150,+0,828,828,+0,828,828,+0,468,468,+0,360,360,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""cx"", 360], [""rz"", 356], [""sx"", 112]]","[[""cx"", 360], [""rz"", 356], [""sx"", 112]]",16,16,2025-10-05T22:22:49
7,ibm_falcon,qaoa,16,r3_seed22_sym,RR→RB,150,150,+0,828,828,+0,828,828,+0,468,468,+0,360,360,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""cx"", 360], [""rz"", 356], [""sx"", 112]]","[[""cx"", 360], [""rz"", 356], [""sx"", 112]]",16,16,2025-10-05T22:22:49
8,ibm_falcon,qaoa,16,r3_seed22_sym,RB→CTM,150,151,+1,828,828,+0,828,828,+0,468,468,+0,360,360,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""cx"", 360], [""rz"", 356], [""sx"", 112]]","[[""cx"", 360], [""rz"", 356], [""sx"", 112]]",16,16,2025-10-05T22:22:49
9,ibm_falcon,qaoa,16,r3_seed22_sym,CTM→RB,150,151,+1,828,828,+0,828,828,+0,468,468,+0,360,360,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""cx"", 360], [""rz"", 356], [""sx"", 112]]","[[""cx"", 360], [""rz"", 356], [""sx"", 112]]",16,16,2025-10-05T22:22:50


,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,quantinuum,qaoa,16,r1_seed22_sym,RB→RR,23,23,+0,108,108,+0,108,108,+0,48,48,+0,60,60,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]","[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]",16,16,2025-10-05T22:22:51
1,quantinuum,qaoa,16,r1_seed22_sym,RR→RB,23,23,+0,108,108,+0,108,108,+0,48,48,+0,60,60,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]","[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]",16,16,2025-10-05T22:22:51
2,quantinuum,qaoa,16,r1_seed22_sym,RB→CTM,23,23,+0,108,108,+0,108,108,+0,48,48,+0,60,60,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]","[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]",16,16,2025-10-05T22:22:51
3,quantinuum,qaoa,16,r1_seed22_sym,CTM→RB,23,23,+0,108,108,+0,108,108,+0,48,48,+0,60,60,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]","[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]",16,16,2025-10-05T22:22:51
4,quantinuum,qaoa,16,r1_seed22_sym,RR→CTM,23,23,+0,108,108,+0,108,108,+0,48,48,+0,60,60,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]","[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]",16,16,2025-10-05T22:22:51
5,quantinuum,qaoa,16,r1_seed22_sym,CTM→RR,23,23,+0,108,108,+0,108,108,+0,48,48,+0,60,60,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]","[[""zzphase"", 60], [""rx"", 32], [""ry"", 16]]",16,16,2025-10-05T22:22:51
6,quantinuum,qaoa,8,r1_seed33_sym,RB→RR,12,12,+0,38,38,+0,38,38,+0,24,24,+0,14,14,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rx"", 16], [""zzphase"", 14], [""ry"", 8]]","[[""rx"", 16], [""zzphase"", 14], [""ry"", 8]]",8,8,2025-10-05T22:22:51
7,quantinuum,qaoa,8,r1_seed33_sym,RR→RB,12,12,+0,38,38,+0,38,38,+0,24,24,+0,14,14,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rx"", 16], [""zzphase"", 14], [""ry"", 8]]","[[""rx"", 16], [""zzphase"", 14], [""ry"", 8]]",8,8,2025-10-05T22:22:51
8,quantinuum,qaoa,8,r1_seed33_sym,RB→CTM,12,12,+0,38,38,+0,38,38,+0,24,24,+0,14,14,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rx"", 16], [""zzphase"", 14], [""ry"", 8]]","[[""rx"", 16], [""zzphase"", 14], [""ry"", 8]]",8,8,2025-10-05T22:22:51
9,quantinuum,qaoa,8,r1_seed33_sym,CTM→RB,12,12,+0,38,38,+0,38,38,+0,24,24,+0,14,14,+0,0,0,+0,0,0,+0,0,0,+0,0,0,+0,"[[""rx"", 16], [""zzphase"", 14], [""ry"", 8]]","[[""rx"", 16], [""zzphase"", 14], [""ry"", 8]]",8,8,2025-10-05T22:22:51


### quick findings

### ibm

the dominant pattern is: **RB ↔ RR** shows essentially no effect; **RB ↔ CTM** produces small, inconsistent depth changes (±1–3); **RR ↔ CTM** shows a directional effect : when **CTM → RR** we typically see a slightly larger depth reduction and consistent decreases in total ops / gates / 1-qubit gates (2-qubit gates unchanged). Changes are small (±1–3 depth on average) but consistent across many seeds/variants. Barriers are absent so not a confound. CTM reorders structure in a way that enables RR to cancel more effectively.

### quantinuum

the pairwise pass-order experiments produced **no measurable changes** across all tested pass combinations (RB, RR, CTM). Depth, total operations, total gates, 1qubit counts, 2qubit counts, and barrier presence are identical for both directions of each pass pair  

**Interpretation:** this suggests one (or more) of the following: 
- the passes are effectively idempotent on Quantinuum-based QAOA circuits
- the generated QAOA circuits contain no redundant single-qubit patterns for these passes to remove
- symbolic parameters prevent algebraic/numeric cancellations that would otherwise create differences  